# ShorelineMonitor: Change Rate

The ShorelineMonitor dataset provides Satellite-Derived Shorelines (SDS) extracted from annually
composited Landsat satellite imagery spanning the years 1984-2024. These shorelines offer a global
view of coastal change and shoreline dynamics, serving as a critical foundation for global coastal
monitoring and analytics. In this notebook we explore the ambient change (trend per transect). 

In [ ]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = coclico_catalog.get_child("gctr")

In [ ]:
snapshot = read_snapshot(collection, storage_options=storage_options)
snapshot.head()

In [ ]:
snapshot.explore()

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = (43.32, -1.97)
m.zoom = 14
m.layout.height = "800px"
m

In [ ]:
west, south, east, north = m.west, m.south, m.east, m.north

# Default values for west, south, east, north (useful for testing or when not set)
if not west:
    west, south, east, north = (4.796, 53.108, 5.229, 53.272)

# Calculate center
center_lon = (west + east) / 2
center_lat = (north + south) / 2

# Print center in copy-pasteable format (rounded to 2 decimal places)
print(f"m.center = ({center_lon:.2f}, {center_lat:.2f})")

# Print extent in copy-pasteable format (rounded to 3 decimal places)
print(f"west, south, east, north = ({west:.3f}, {south:.3f}, {east:.3f}, {north:.3f})")

roi = gpd.GeoDataFrame(
    geometry=[shapely.geometry.box(west, south, east, north)], crs=4326
)

## Dashboard

In [ ]:
import coastpy

db = coastpy.io.STACQueryEngine(
    stac_collection=gctr_col,
    storage_backend="azure",
    # columns = ["geometry", "transect_id", "sds:change_rate"] ... # when you don't need all data
)

In [ ]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
print(f"Shape: {df.shape}")
df.head()

In [ ]:
ac = gpd.GeoDataFrame(
    gctr[["transect_id", "sds:change_rate"]],
    geometry=gpd.GeoSeries.from_xy(gctr.lon, gctr.lat, crs=4326),
)

In [ ]:
import hvplot.pandas  
import colorcet as cc

def plot_shoreline_change_trend(ac_gdf):
    """
    Simple hvplot visualization for shoreline change trends.
    """
    # Calculate the 2.5th and 97.5th percentiles to reduce extreme outliers
    lower_quantile = ac_gdf["sds:change_rate"].quantile(0.025)
    upper_quantile = ac_gdf["sds:change_rate"].quantile(0.975)

    # Define symmetric color limits around zero for clear erosion/accretion visualization
    max_abs_range = max(abs(lower_quantile), abs(upper_quantile))
    clim = (-max_abs_range, max_abs_range)
    
    plot = ac_gdf.hvplot(
        geo=True,
        tiles="EsriImagery",  
        color="sds:change_rate",
        cmap=cc.CET_D3[::-1],  
        clim=clim,
        frame_width=800,
        frame_height=600,
        colorbar=True,
        title="Shoreline Change Trend (m/year)",
        tools=["hover"],
    )
    
    return plot

In [ ]:
plot_shoreline_change_trend(ac)